In [1]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import time

# Thiết lập thiết bị sử dụng GPU nếu có, ngược lại dùng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 1. Định nghĩa mạng neural cho Actor-Critic ---
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=64):
        super(ActorCritic, self).__init__()
        # Mạng dùng chung
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.Tanh(),
        )
        # Actor: Xuất xác suất cho từng hành động (dành cho không gian hành động rời rạc)
        self.policy = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, action_dim),
            nn.Softmax(dim=-1)
        )
        # Critic: Ước lượng giá trị trạng thái
        self.value = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        
    def forward(self, state):
        x = self.shared(state)
        action_probs = self.policy(x)
        state_value = self.value(x)
        return action_probs, state_value

# --- 2. Hyperparameters ---
env_name = "CartPole-v1"
env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

hidden_dim = 64
lr = 3e-4
gamma = 0.99            # Hệ số chiết khấu
lmbda = 0.95            # Hệ số lambda trong GAE
clip_epsilon = 0.2      # Hệ số clipping của PPO
ppo_epochs = 10         # Số epoch cập nhật mỗi vòng
minibatch_size = 64
max_timesteps_per_update = 2048
total_updates = 50

# Hệ số loss
vf_coef = 0.5           # Hệ số cho Value Function loss
entropy_coef = 0.01     # Hệ số cho Entropy bonus

# --- 3. Khởi tạo mạng và optimizer ---
model = ActorCritic(state_dim, action_dim, hidden_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

# --- 4. Hàm tính Generalized Advantage Estimation (GAE) ---
def compute_gae(rewards, masks, values, gamma, lmbda):
    # Chuyển list values thành tensor
    values = torch.tensor(values, dtype=torch.float32, device=device)
    gae = 0
    advantages = []
    # Duyệt ngược từ cuối về đầu (len(rewards) = T)
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * lmbda * masks[step] * gae
        advantages.insert(0, gae)
    advantages = torch.tensor(advantages, dtype=torch.float32, device=device)
    # Value target = advantage + value (lấy từ values từ 0 đến T-1)
    returns = advantages + values[:-1]
    return advantages, returns

# --- 5. Hàm thu thập dữ liệu từ môi trường ---
def collect_trajectories(env, model, max_timesteps):
    states = []
    actions = []
    rewards = []
    masks = []       # 1 nếu episode chưa kết thúc, 0 nếu kết thúc
    log_probs = []
    values = []
    
    # Với gymnasium (gym >=0.26), reset trả về (observation, info)
    state, _ = env.reset()
    for _ in range(max_timesteps):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        probs, value = model(state_tensor)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        
        # Lưu dữ liệu
        states.append(state)
        actions.append(action.item())
        values.append(value.item())
        log_probs.append(log_prob.item())
        
        # Thực hiện hành động: gymnasium step trả về (observation, reward, done, truncated, info)
        next_state, reward, done, truncated, _ = env.step(action.item())
        rewards.append(reward)
        masks.append(0 if (done or truncated) else 1)
        
        state = next_state
        if done or truncated:
            state, _ = env.reset()
    # Thêm giá trị của trạng thái cuối để tính advantage (sử dụng giá trị của state cuối cùng)
    state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
    _, next_value = model(state_tensor)
    values.append(next_value.item())
    
    return states, actions, rewards, masks, log_probs, values

# --- 6. Hàm cập nhật PPO ---
def ppo_update(model, optimizer, states, actions, log_probs_old, returns, advantages, clip_epsilon, ppo_epochs, minibatch_size):
    # Chuyển dữ liệu thành tensor nếu cần
    if not isinstance(states, torch.Tensor):
        states = torch.FloatTensor(states).to(device)
    if not isinstance(actions, torch.Tensor):
        actions = torch.LongTensor(actions).to(device)
    if not isinstance(log_probs_old, torch.Tensor):
        log_probs_old = torch.FloatTensor(log_probs_old).to(device)
    if not isinstance(returns, torch.Tensor):
        returns = torch.tensor(returns, dtype=torch.float32).to(device)
    else:
        returns = returns.to(device)
    if not isinstance(advantages, torch.Tensor):
        advantages = torch.tensor(advantages, dtype=torch.float32).to(device)
    else:
        advantages = advantages.to(device)
    
    dataset_size = states.size(0)
    for _ in range(ppo_epochs):
        indices = np.arange(dataset_size)
        np.random.shuffle(indices)
        for i in range(0, dataset_size, minibatch_size):
            # Chuyển chỉ số sang tensor để index tensor
            batch_idx = torch.tensor(indices[i: i + minibatch_size], dtype=torch.long, device=device)
            batch_states = states[batch_idx]
            batch_actions = actions[batch_idx]
            batch_log_probs_old = log_probs_old[batch_idx]
            batch_returns = returns[batch_idx]
            batch_advantages = advantages[batch_idx]
            
            # Tính toán xác suất và giá trị mới từ model
            probs, values = model(batch_states)
            dist = Categorical(probs)
            log_probs = dist.log_prob(batch_actions)
            entropy = dist.entropy().mean()
            
            # Tỷ lệ cập nhật
            ratio = torch.exp(log_probs - batch_log_probs_old)
            
            # Tính surrogate objective với clipping
            surr1 = ratio * batch_advantages
            surr2 = torch.clamp(ratio, 1.0 - clip_epsilon, 1.0 + clip_epsilon) * batch_advantages
            policy_loss = -torch.min(surr1, surr2).mean()
            
            # Value function loss (Mean Squared Error)
            value_loss = (batch_returns - values.squeeze()).pow(2).mean()
            
            # Tổng loss, bao gồm entropy bonus để khuyến khích khám phá
            loss = policy_loss + vf_coef * value_loss - entropy_coef * entropy
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

# --- 7. Vòng lặp huấn luyện chính của PPO ---
all_episode_rewards = []
for update in range(total_updates):
    states, actions, rewards, masks, log_probs_old, values = collect_trajectories(env, model, max_timesteps_per_update)
    
    # Tính advantage và returns sử dụng GAE
    advantages, returns = compute_gae(rewards, masks, values, gamma, lmbda)
    
    # Cập nhật PPO qua nhiều epoch trên minibatch
    ppo_update(model, optimizer, states, actions, log_probs_old, returns, advantages, clip_epsilon, ppo_epochs, minibatch_size)
    
    # Tính toán và in ra phần thưởng trung bình của các episode trong batch
    if update % 10 == 0:
        # Giả sử mỗi episode có tối đa 500 bước, ta tính số episode = len(rewards)/500
        num_episodes = len(rewards) / 500
        avg_reward = np.sum(rewards) / (num_episodes if num_episodes > 0 else 1)
        all_episode_rewards.append(avg_reward)
        print(f'Update {update}, Average Reward: {avg_reward:.2f}')

# --- 8. Sau khi huấn luyện, visualize hoạt ảnh của agent ---
# Chạy một vài episode và render hoạt ảnh
for ep in range(5):
    state, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        env.render()  # Hiển thị hoạt ảnh
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            probs, _ = model(state_tensor)
        action = torch.argmax(probs, dim=1).item()  # Chọn hành động có xác suất cao nhất
        next_state, reward, done, truncated, _ = env.step(action)
        total_reward += reward
        state = next_state
        time.sleep(0.02)
        if done or truncated:
            break
    print(f'Episode {ep+1}: Total Reward: {total_reward}')
env.close()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\nguye\anaconda3\envs\ai\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\nguye\anaconda3\envs\ai\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\nguye\anaconda3\envs\ai\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\Users\nguye

Update 0, Average Reward: 500.00
Update 10, Average Reward: 500.00
Update 20, Average Reward: 500.00
Update 30, Average Reward: 500.00
Update 40, Average Reward: 500.00


c:\Users\nguye\anaconda3\envs\ai\Lib\site-packages\gymnasium\envs\classic_control\cartpole.py:250: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


Episode 1: Total Reward: 500.0
Episode 2: Total Reward: 500.0
Episode 3: Total Reward: 500.0
Episode 4: Total Reward: 500.0
Episode 5: Total Reward: 500.0
